# 7 — Image Processing with OpenCV in ROS2

> Use `cv_bridge` to convert ROS images, then apply OpenCV filters, edge detection, and colour tracking.

**Goal:** Build a complete image processing pipeline as a ROS2 node: receive camera images, process with OpenCV, and republish the result.

In [ ]:
#| default_exp opencv_processing

## 7.1 The Bridge: ROS Images ↔ OpenCV

ROS2 uses `sensor_msgs/msg/Image` for images. OpenCV uses **NumPy arrays**. **`cv_bridge`** converts between them:

```
sensor_msgs/Image  ──cv_bridge──►  numpy.ndarray (OpenCV)
  (ROS message)       .imgmsg_to_cv2()    (H × W × C array)

numpy.ndarray      ──cv_bridge──►  sensor_msgs/Image
  (OpenCV)            .cv2_to_imgmsg()    (ROS message)
```

### Installation Check

```bash
# Should already be installed (notebook 01)
python3 -c "from cv_bridge import CvBridge; print('cv_bridge OK')"
python3 -c "import cv2; print(f'OpenCV {cv2.__version__} OK')"
```

## 7.2 cv_bridge Basics

```python
from cv_bridge import CvBridge
import cv2
import numpy as np

bridge = CvBridge()

# ROS Image → OpenCV (numpy)
cv_image = bridge.imgmsg_to_cv2(ros_image_msg, desired_encoding='bgr8')
# cv_image is now a numpy array of shape (480, 640, 3)

# OpenCV (numpy) → ROS Image
ros_image_msg = bridge.cv2_to_imgmsg(cv_image, encoding='bgr8')
```

### Common Encodings

| Encoding | Channels | Used for |
|----------|----------|----------|
| `bgr8` | 3 | Standard OpenCV colour (Blue-Green-Red) |
| `rgb8` | 3 | Standard colour (Red-Green-Blue) |
| `mono8` | 1 | Grayscale |
| `32FC1` | 1 (float) | Depth maps |
| `passthrough` | auto | Use whatever the source has |

> **Important:** OpenCV uses **BGR** by default, not RGB. When converting, use `desired_encoding='bgr8'`.

## 7.3 The Processing Node Pattern

Every image processing node follows this pattern:

```
Subscribe to input image → Convert → Process → Convert back → Publish
```

```
/camera/image_raw  →  [Your Node]  →  /camera/processed
```

## 7.4 Example 1: Grayscale Converter

In [ ]:
#| export

GRAYSCALE_NODE = '''
#!/usr/bin/env python3
"""Converts camera images to grayscale using OpenCV."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2


class GrayscaleNode(Node):
    def __init__(self):
        super().__init__('grayscale_node')

        self.bridge = CvBridge()

        # Subscribe to raw camera images
        self.subscription = self.create_subscription(
            Image,
            '/camera/image_raw',
            self.image_callback,
            qos_profile_sensor_data
        )

        # Publish processed images on a new topic
        self.publisher_ = self.create_publisher(
            Image,
            '/camera/grayscale',
            10
        )

        self.get_logger().info('GrayscaleNode ready')

    def image_callback(self, msg: Image):
        # 1. Convert ROS Image → OpenCV
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # 2. Process with OpenCV
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)

        # 3. Convert OpenCV → ROS Image
        gray_msg = self.bridge.cv2_to_imgmsg(gray, encoding='mono8')
        gray_msg.header = msg.header  # preserve timestamp and frame

        # 4. Publish
        self.publisher_.publish(gray_msg)


def main(args=None):
    rclpy.init(args=args)
    node = GrayscaleNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(GRAYSCALE_NODE)

### What's Happening

1. **Subscribe** to `/camera/image_raw` with sensor QoS
2. **Convert** the ROS message to a NumPy array (BGR format)
3. **Process** — convert BGR → Grayscale using `cv2.cvtColor`
4. **Convert back** to a ROS Image message
5. **Copy the header** — this preserves the timestamp and frame_id (important for synchronization)
6. **Publish** on `/camera/grayscale`

## 7.5 Example 2: Edge Detection (Canny)

In [ ]:
#| export

EDGE_DETECTION_NODE = '''
#!/usr/bin/env python3
"""Detects edges in camera images using Canny edge detection."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2


class EdgeDetectionNode(Node):
    def __init__(self):
        super().__init__('edge_detection_node')

        # Declare configurable parameters
        self.declare_parameter('low_threshold', 50)
        self.declare_parameter('high_threshold', 150)
        self.declare_parameter('blur_kernel_size', 5)

        self.bridge = CvBridge()

        self.subscription = self.create_subscription(
            Image, '/camera/image_raw',
            self.image_callback, qos_profile_sensor_data
        )
        self.publisher_ = self.create_publisher(Image, '/camera/edges', 10)

        self.get_logger().info('EdgeDetectionNode ready')

    def image_callback(self, msg: Image):
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Read parameters (allows runtime tuning)
        low = self.get_parameter('low_threshold').value
        high = self.get_parameter('high_threshold').value
        ksize = self.get_parameter('blur_kernel_size').value

        # Convert to grayscale
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)

        # Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray, (ksize, ksize), 0)

        # Canny edge detection
        edges = cv2.Canny(blurred, low, high)

        # Publish
        edge_msg = self.bridge.cv2_to_imgmsg(edges, encoding='mono8')
        edge_msg.header = msg.header
        self.publisher_.publish(edge_msg)


def main(args=None):
    rclpy.init(args=args)
    node = EdgeDetectionNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(EDGE_DETECTION_NODE)

### Tune at Runtime!

```bash
# While the node is running, adjust thresholds:
ros2 param set /edge_detection_node low_threshold 30
ros2 param set /edge_detection_node high_threshold 100

# See the effect in rqt_image_view on /camera/edges
```

This is the power of combining ROS2 parameters with OpenCV — **live tuning without restart**.

## 7.6 Example 3: Colour Object Tracking

In [ ]:
#| export

COLOR_TRACKER_NODE = '''
#!/usr/bin/env python3
"""Tracks a coloured object (default: red) and draws a bounding circle."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from geometry_msgs.msg import Point
from cv_bridge import CvBridge
import cv2
import numpy as np


class ColorTrackerNode(Node):
    def __init__(self):
        super().__init__('color_tracker_node')

        # HSV range for red colour detection
        # Red wraps around in HSV, so we need two ranges
        self.declare_parameter('h_low_1', 0)
        self.declare_parameter('h_high_1', 10)
        self.declare_parameter('h_low_2', 170)
        self.declare_parameter('h_high_2', 180)
        self.declare_parameter('s_low', 100)
        self.declare_parameter('s_high', 255)
        self.declare_parameter('v_low', 100)
        self.declare_parameter('v_high', 255)
        self.declare_parameter('min_area', 500)  # minimum contour area

        self.bridge = CvBridge()

        self.subscription = self.create_subscription(
            Image, '/camera/image_raw',
            self.image_callback, qos_profile_sensor_data
        )

        # Publish annotated image
        self.image_pub = self.create_publisher(Image, '/camera/tracked', 10)
        # Publish object position
        self.position_pub = self.create_publisher(Point, '/tracked_object/position', 10)

        self.get_logger().info('ColorTrackerNode ready — tracking red objects')

    def image_callback(self, msg: Image):
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Convert to HSV colour space
        hsv = cv2.cvtColor(cv_image, cv2.COLOR_BGR2HSV)

        # Read parameters
        s_low = self.get_parameter('s_low').value
        s_high = self.get_parameter('s_high').value
        v_low = self.get_parameter('v_low').value
        v_high = self.get_parameter('v_high').value

        # Create masks for red (two HSV ranges)
        lower1 = np.array([self.get_parameter('h_low_1').value, s_low, v_low])
        upper1 = np.array([self.get_parameter('h_high_1').value, s_high, v_high])
        lower2 = np.array([self.get_parameter('h_low_2').value, s_low, v_low])
        upper2 = np.array([self.get_parameter('h_high_2').value, s_high, v_high])

        mask1 = cv2.inRange(hsv, lower1, upper1)
        mask2 = cv2.inRange(hsv, lower2, upper2)
        mask = mask1 | mask2

        # Clean up the mask
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv2.findContours(
            mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        min_area = self.get_parameter('min_area').value

        if contours:
            # Find the largest contour
            largest = max(contours, key=cv2.contourArea)
            area = cv2.contourArea(largest)

            if area > min_area:
                # Get enclosing circle
                ((cx, cy), radius) = cv2.minEnclosingCircle(largest)

                # Draw on the image
                cv2.circle(cv_image, (int(cx), int(cy)), int(radius), (0, 255, 0), 2)
                cv2.circle(cv_image, (int(cx), int(cy)), 5, (0, 0, 255), -1)
                cv2.putText(
                    cv_image,
                    f'({int(cx)}, {int(cy)}) area={int(area)}',
                    (int(cx) - 50, int(cy) - int(radius) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2
                )

                # Publish position
                pos = Point()
                pos.x = cx
                pos.y = cy
                pos.z = float(radius)  # use z for radius
                self.position_pub.publish(pos)

        # Publish annotated image
        out_msg = self.bridge.cv2_to_imgmsg(cv_image, encoding='bgr8')
        out_msg.header = msg.header
        self.image_pub.publish(out_msg)


def main(args=None):
    rclpy.init(args=args)
    node = ColorTrackerNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(COLOR_TRACKER_NODE)

### How Colour Tracking Works

```
BGR Image  →  HSV Conversion  →  Colour Mask  →  Morphology  →  Contours  →  Largest  →  Draw
```

1. **HSV Conversion** — HSV (Hue, Saturation, Value) is better for colour detection than BGR because *hue* represents the colour regardless of lighting
2. **`inRange`** — Creates a binary mask (white = matches colour, black = doesn't)
3. **Morphology** — Removes small noise (OPEN = erode+dilate) and fills gaps (CLOSE = dilate+erode)
4. **Contours** — Finds the outlines of white regions in the mask
5. **Largest contour** — The biggest blob is our tracked object

### Tuning HSV Values

Use `rqt_reconfigure` or `ros2 param set` to tune in real time:

```bash
# To track BLUE instead of red:
ros2 param set /color_tracker_node h_low_1 100
ros2 param set /color_tracker_node h_high_1 130
ros2 param set /color_tracker_node h_low_2 100  # same range (blue doesn't wrap)
ros2 param set /color_tracker_node h_high_2 130
```

Common HSV ranges:

| Colour | H range |
|--------|--------|
| Red | 0-10, 170-180 |
| Orange | 10-25 |
| Yellow | 25-35 |
| Green | 35-85 |
| Blue | 100-130 |
| Purple | 130-170 |

## 7.7 Example 4: Multi-Processing Pipeline

In [ ]:
#| export

PIPELINE_NODE = '''
#!/usr/bin/env python3
"""Image pipeline: applies multiple OpenCV operations in sequence."""

import rclpy
from rclpy.node import Node
from rclpy.qos import qos_profile_sensor_data
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2
import numpy as np
import time


class ImagePipelineNode(Node):
    def __init__(self):
        super().__init__('image_pipeline_node')

        self.declare_parameter('enable_blur', True)
        self.declare_parameter('enable_edges', False)
        self.declare_parameter('enable_histogram_eq', True)
        self.declare_parameter('show_fps', True)

        self.bridge = CvBridge()
        self.prev_time = time.time()
        self.fps = 0.0

        self.subscription = self.create_subscription(
            Image, '/camera/image_raw',
            self.image_callback, qos_profile_sensor_data
        )
        self.publisher_ = self.create_publisher(Image, '/camera/processed', 10)

        self.get_logger().info('ImagePipelineNode ready')

    def image_callback(self, msg: Image):
        cv_image = self.bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # Step 1: Optional Gaussian Blur
        if self.get_parameter('enable_blur').value:
            cv_image = cv2.GaussianBlur(cv_image, (5, 5), 0)

        # Step 2: Optional Histogram Equalization
        if self.get_parameter('enable_histogram_eq').value:
            # Convert to LAB, equalize L channel, convert back
            lab = cv2.cvtColor(cv_image, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            l = clahe.apply(l)
            lab = cv2.merge([l, a, b])
            cv_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

        # Step 3: Optional Edge Detection
        if self.get_parameter('enable_edges').value:
            gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
            edges = cv2.Canny(gray, 50, 150)
            cv_image = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

        # FPS overlay
        if self.get_parameter('show_fps').value:
            now = time.time()
            self.fps = 0.9 * self.fps + 0.1 * (1.0 / max(now - self.prev_time, 1e-6))
            self.prev_time = now
            cv2.putText(
                cv_image, f'FPS: {self.fps:.1f}',
                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2
            )

        # Publish
        out_msg = self.bridge.cv2_to_imgmsg(cv_image, encoding='bgr8')
        out_msg.header = msg.header
        self.publisher_.publish(out_msg)


def main(args=None):
    rclpy.init(args=args)
    node = ImagePipelineNode()
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        node.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()
'''

print(PIPELINE_NODE)

### Toggle Stages at Runtime

```bash
# Enable edge detection
ros2 param set /image_pipeline_node enable_edges true

# Disable blur
ros2 param set /image_pipeline_node enable_blur false
```

## 7.8 Complete Launch File

In [ ]:
#| export

VISION_LAUNCH = '''
#!/usr/bin/env python3
"""Launch camera + OpenCV processing pipeline."""

from launch import LaunchDescription
from launch_ros.actions import Node


def generate_launch_description():
    return LaunchDescription([
        # Camera driver
        Node(
            package='usb_cam',
            executable='usb_cam_node_exe',
            name='camera',
            parameters=[{
                'video_device': '/dev/video0',
                'image_width': 640,
                'image_height': 480,
                'framerate': 30.0,
            }],
            remappings=[
                ('image_raw', '/camera/image_raw'),
            ]
        ),

        # Edge detection node
        Node(
            package='robot_vision',
            executable='edge_detection_node',
            name='edge_detection_node',
            parameters=[{
                'low_threshold': 50,
                'high_threshold': 150,
            }]
        ),

        # Colour tracker node
        Node(
            package='robot_vision',
            executable='color_tracker_node',
            name='color_tracker_node',
        ),
    ])
'''

print(VISION_LAUNCH)

## 7.9 Performance Tips

| Tip | Why |
|-----|-----|
| **Reduce resolution** (320×240 instead of 640×480) | 4× fewer pixels = 4× faster |
| **Skip frames** — process every Nth frame | If CV is slower than camera rate |
| **Use ROI** (Region of Interest) | Only process a sub-region |
| **Use `cv2.UMat`** for GPU-accelerated OpenCV | If you have OpenCL support |
| **Publish less** — only annotated results, not intermediate | Reduces topic bandwidth |

### Frame Skipping Pattern

```python
def image_callback(self, msg):
    self.frame_count += 1
    if self.frame_count % 3 != 0:  # process every 3rd frame
        return
    # ... heavy processing ...
```

## 7.10 OpenCV Quick Reference

Common operations you'll use in ROS2 vision nodes:

| Operation | Code |
|-----------|------|
| Resize | `cv2.resize(img, (320, 240))` |
| Crop ROI | `roi = img[y1:y2, x1:x2]` |
| Colour convert | `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)` |
| Blur | `cv2.GaussianBlur(img, (5,5), 0)` |
| Threshold | `cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)` |
| Canny edges | `cv2.Canny(gray, 50, 150)` |
| Find contours | `cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)` |
| Draw circle | `cv2.circle(img, (cx, cy), r, (0,255,0), 2)` |
| Draw rectangle | `cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)` |
| Put text | `cv2.putText(img, 'text', (x,y), font, scale, color, thickness)` |
| Morphology | `cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)` |

---

**Next →** [Notebook 08: Deep Learning in ROS2](08_deep_learning.ipynb)